# Hadoop Streaming assignment 1: Words Rating

The purpose of this task is to create your own WordCount program for Wikipedia dump processing and learn basic concepts of the MapReduce.

In this task you have to find the 7th word by popularity and its quantity in the reverse order (most popular first) in Wikipedia data (`/data/wiki/en_articles_part`).

There are several points for this task:

1) As an output, you have to get the 7th word and its quantity separated by a tab character.

2) You must use the second job to obtain a totally ordered result.

3) Do not forget to redirect all trash and output to /dev/null.

Here you can find the draft of the task main steps. You can use other methods for solution obtaining.

## Step 1. Create mapper and reducer.

<b>Hint:</b>  Demo task contains almost all the necessary pieces to complete this assignment. You may use the demo to implement the first MapReduce Job.

In [1]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)
        

Writing mapper1.py


In [2]:
%%writefile reducer1.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Writing reducer1.py


In [3]:
# You can use this cell for other experiments: for example, for combiner.

## Step 2. Create sort job.

<b>Hint:</b> You may use MapReduce comparator to solve this step. Make sure that the keys are sorted in ascending order.

In [27]:
%%writefile mapper2.py
# Your code for sort job here. Don't forget to use m%%writefile mapper2.py

import sys

reload(sys)
sys.setdefaultencoding('utf-8')

lines = []

for line in sys.stdin:
    
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
        lines.append((key, count))
    except ValueError as e:
        continue
    
sorted_pairs = sorted(lines, key=lambda k: k[1], reverse=True)
    
for pair in sorted_pairs:
    print "%s\t%d" % (pair[0], pair[1])
    


#for line in sys.stdin:
    
#    try:
#        key, count = line.strip().split('\t', 1)
#        count = int(count)
#        print "%s\t%d" % (key, count)
#    except ValueError as e:
#        continue



Overwriting mapper2.py


In [15]:
%%writefile reducer2.py
import sys

reload(sys)
sys.setdefaultencoding('utf-8')

#lines = []

#for line in sys.stdin:
    
#    try:
#        key, count = line.strip().split('\t', 1)
#        count = int(count)
#        lines.append((key, count))
#    except ValueError as e:
#        continue
    
#sorted_pairs = sorted(lines, key=lambda k: k[1], reverse=True)
    
#for pair in sorted_pairs:
#    print "%s.%d" % (pair[0], pair[1])
    


#for line in sys.stdin:
    
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
        print "%s\t%d" % (key, count)
    except ValueError as e:
        continue

    

Overwriting reducer2.py


## Step 3. Bash commands

<b> Hint: </b> For printing the exact row you may use basic UNIX commands. For instance, sed/head/tail/... (if you know other commands, you can use them).

To run both jobs, you must use two consecutive yarn-commands. Remember that the input for the second job is the ouput for the first job.

In [39]:
%%bash

OUT_DIR="assignment1_"$(date +"%s%6N")
#OUT_DIR="assignment1_1549016700915885"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null
# Code for your first job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -combiner "python reducer1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null
    
OUT_DIR2="assignment1_final_"$(date +"%s%6N")
# Code for your second job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2nr" \
    -D mapred.jab.name="Streaming wordCount 2" \
    -D mapreduce.job.reduces=1 \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python mapper2.py" \
    -input ${OUT_DIR} -output ${OUT_DIR2} > /dev/null

    #-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    #-D mapred.text.key.comparator.options="-k2nr" \

    #-D mapreduce.map.output.key.field.separator=. \
    #-D mapreduce.partition.keycomparator.options="-k2nr" \
    #-D stream.map.output.field.separator=. \
    #-D stream.num.map.output.key.fields=1 \



# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR2}/part-00000 | sed -n '7p;8q'
hdfs dfs -rm -r -skipTrash ${OUT_DIR}* > /dev/null

is	126420


rm: `assignment1_1549018965354308': No such file or directory
19/02/01 11:02:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/01 11:02:49 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/01 11:02:49 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/01 11:02:49 INFO mapreduce.JobSubmitter: number of splits:2
19/02/01 11:02:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549016149173_0018
19/02/01 11:02:50 INFO impl.YarnClientImpl: Submitted application application_1549016149173_0018
19/02/01 11:02:50 INFO mapreduce.Job: The url to track the job: http://9a1c3d10e2c4:8088/proxy/application_1549016149173_0018/
19/02/01 11:02:50 INFO mapreduce.Job: Running job: job_1549016149173_0018
19/02/01 11:02:55 INFO mapreduce.Job: Job job_1549016149173_0018 running in uber mode : false
19/02/01 11:02:55 INFO mapreduce.Job:  map 0% reduce 0%
19/02/01 11:03:11 INFO mapreduce.Job:  map 33% reduce 0%
19/02/01 11:03:17 

the	822164
of	447464
and	342715
in	292354
to	241467
a	236225
is	126420
as	103301
for	91245
was	90336
by	85781
that	81572
with	79559
on	71710
from	58201
are	57416
it	49620
or	49404
an	46431
his	45001
cat: Unable to write to output stream.


monopolise	2
monopolised	2
monopolists	2
monostatic	2
monoxide's	2
montand's	2
month_type	2
moon’s	2
moor's	2
monkeemania	2
monologist	2
mootz	2
moraxella	2
moinuddin	2
moiré	2
moisture/density	2
mol/kg	2
molecule(s	2
molinos	2
mondsee	2
cat: Unable to write to output stream.
